In [11]:
import pandas as pd

SENTENCES_TO_KEEP = 2
OUTPUT_PATH = 'Augmented Datasets/'

In [12]:
from datasets import load_dataset

fever_plus = load_dataset("tommasobonomo/sem_augmented_fever_nli")

In [13]:
fever_plus["train"]

Dataset({
    features: ['id', 'premise', 'hypothesis', 'label', 'wsd', 'srl'],
    num_rows: 51086
})

In [14]:
i = 0
fever_plus["train"][i]['wsd']['premise'][0]['lemma']
# for i in range(len(fever_plus["train"])):
#     print(get_first_N_sentences(fever_plus["train"]['premise'][i], 1))

'roman'

In [15]:
for i in range(100):
    print(fever_plus["train"][i]["premise"])

Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a daily basis . His vlogging channel , `` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks .
Boston Celtics . The Celtics play their home games at the TD Garden , which they share with the National Hockey League ( NHL ) 's Boston Bruins .
The Hunger Games is a 2012 American dystopian science fiction adventure film directed by Gary Ross and based on the novel of the same name by Suzanne Collins . The film stars Jennifer Lawrence , Josh Hutcherson , Liam Hemsworth , Woody Harrelson , Elizabeth Banks , Lenny Kravitz , Stanley Tucci , and Donald Sutherland .
Ryan John Seacrest ( born December 24 , 1974 ) is an American radio personality , television host and producer . Seacrest began co-hosting Live with Kelly and Ryan on a permanent basis May 1 , 2017 . He received Emmy Award nomin

In [16]:
fever_plus["train"][90]['wsd']['hypothesis']

[{'index': 0,
  'text': 'The',
  'pos': 'DET',
  'lemma': 'the',
  'bnSynsetId': 'O',
  'wnSynsetOffset': 'O',
  'nltkSynset': 'O'},
 {'index': 1,
  'text': 'Challenge',
  'pos': 'PROPN',
  'lemma': 'Challenge',
  'bnSynsetId': 'O',
  'wnSynsetOffset': 'O',
  'nltkSynset': 'O'},
 {'index': 2,
  'text': 'was',
  'pos': 'AUX',
  'lemma': 'be',
  'bnSynsetId': 'O',
  'wnSynsetOffset': 'O',
  'nltkSynset': 'O'},
 {'index': 3,
  'text': 'a',
  'pos': 'DET',
  'lemma': 'a',
  'bnSynsetId': 'O',
  'wnSynsetOffset': 'O',
  'nltkSynset': 'O'},
 {'index': 4,
  'text': 'reality',
  'pos': 'NOUN',
  'lemma': 'reality',
  'bnSynsetId': 'bn:00066419n',
  'wnSynsetOffset': '13956097n',
  'nltkSynset': 'reality.n.03'},
 {'index': 5,
  'text': 'game',
  'pos': 'NOUN',
  'lemma': 'game',
  'bnSynsetId': 'bn:00037183n',
  'wnSynsetOffset': '2152991n',
  'nltkSynset': 'game.n.04'},
 {'index': 6,
  'text': 'show',
  'pos': 'NOUN',
  'lemma': 'show',
  'bnSynsetId': 'bn:00071316n',
  'wnSynsetOffset': '6619

In [17]:
import wikipedia

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# get the summary of the wikipedia page with "page_title"; returns an empty string if nothing is found until now
def get_wikipedia_summary(page_title):
    if not page_title:  # if passed a None for entry, return nothing
        return ''
    try:
        summary = wikipedia.summary(page_title)
        return summary
    except wikipedia.DisambiguationError as e:
        return ''     # f"Disambiguation error: {e.options}"
    except wikipedia.PageError:
        return ''     # f"No page found for {page_title}"

# get N sentences from wikipedia summary or any text in general   
def get_first_N_sentences(text, N):
    sentences = sent_tokenize(text)
    text_shortened = sentences[:N]
    return ' '.join(text_shortened)


# Example usage
page_title = "Python (programming language)"
summary = get_wikipedia_summary(page_title)
summary = get_first_N_sentences(summary, SENTENCES_TO_KEEP)
print(summary)


Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\damic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# take the first PROPN words N-gram tagged (to be used to search wikipedia later)
def retrieve_pag_to_search(annotated_sentence):   # remember to pass something like: fever_plus['train'][elem]

    page_to_search = ''
    last_was_PROPN = False

    for word in annotated_sentence['wsd']['hypothesis']:  # search the sentence word by word...
        if word['pos'] == 'PROPN':      # ...if a proper name is found, update the page to search...
           page_to_search += ' ' + word['text']
           last_was_PROPN = True
        elif last_was_PROPN:            # ...else if not, stop the search beacuse the PROPN N-gram is finished
            break

    return page_to_search

In [19]:
import wikipedia
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def get_wikipedia_sense(word, sentence):
    # Search for the word in Wikipedia
    try:
        search_results = wikipedia.search(word)
    except wikipedia.WikipediaException:
        return None  # Return None if there's an error during search

    if not search_results:  # If no page entries are found
        return None

    # Create a TF-IDF vectorizer and fit it on the sentence and search results
    vectorizer = TfidfVectorizer().fit_transform([sentence] + search_results)
    vectors = vectorizer.toarray()
    
    # Compute cosine similarities between the sentence and the search results
    cosine_similarities = cosine_similarity(vectors[0:1], vectors[1:]).flatten()
    best_match_index = cosine_similarities.argmax()

    # Obtain the best corresponding page if applicable, else None
    try:
        best_page_title = search_results[best_match_index]
        best_page = wikipedia.page(best_page_title)
        return best_page
    except wikipedia.DisambiguationError as e:
        # Handling disambiguation by checking if there are options
        if e.options:
            # Optionally, you can further refine this to select the most relevant option
            for option in e.options:
                try:
                    option_page = wikipedia.page(option)
                    return option_page
                except wikipedia.WikipediaException:    # should be PageError, but to be sure skip the word if any error occur (I'm looking at you, 'Portugual' wiki entry)
                    continue
        return None
    except wikipedia.PageError:
        return None
    except wikipedia.WikipediaException:
        return None  # Handle other Wikipedia exceptions

# Example of usage
word = "jaguar"
sentence = "The jaguar is a large cat species and the only extant member of the genus Panthera native to the Americas."
sense = get_wikipedia_sense(word, sentence)
print(sense)


<WikipediaPage 'Jaguar'>


In [20]:

#############################
# FULL AUGMENT LOOP:        #
# WIKIPEDIA REDUDANCY       #
#############################

from tqdm import tqdm
# Suppress specific warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="wikipedia")

# Wikipedia Redudancy aumenter function for a single dataset
def WR_augment(dataset, sentences_to_keep):
  wr_list = []

  for entry in tqdm(dataset, desc="Augmenting Dataset"):
      
      page_to_search = retrieve_pag_to_search(entry)
      disambiguated_page = get_wikipedia_sense(page_to_search, entry['premise'])
      summary = get_first_N_sentences(get_wikipedia_summary(disambiguated_page), sentences_to_keep)
      # print(summary)

      # add entry to list if found, otherwise add ''
      wr_list.append({
                      'id' : entry['id'],
                      'summary' : summary,
                      'old_premise' : entry['premise'], 
                      'augmented_premise' : summary + ' ' + entry['premise'], 
                      'hypothesis': entry['hypothesis'], 
                      'label' : entry['label'], 
                      'wsd' : entry['wsd'], 
                      'srl' : entry['srl']
                    })  

  return wr_list



# WR-Augmented datasets creation
wr_augmented_fever_train = pd.DataFrame(WR_augment(fever_plus['train'], SENTENCES_TO_KEEP))
print('train dataset done')
wr_augmented_fever_eval = pd.DataFrame(WR_augment(fever_plus['eval'], SENTENCES_TO_KEEP))
print('eval dataset done')
wr_augmented_fever_test = pd.DataFrame(WR_augment(fever_plus['test'], SENTENCES_TO_KEEP))
print('test dataset done')


Augmenting Dataset:   0%|          | 67/51086 [02:35<30:51:11,  2.18s/it]

In [ ]:
# Write DataFrames to jsonl files
wr_augmented_fever_train.to_json(OUTPUT_PATH + 'fever_train_wr.jsonl', orient='records', lines=True)
wr_augmented_fever_eval.to_json(OUTPUT_PATH + 'fever_eval_wr.jsonl', orient='records', lines=True)
wr_augmented_fever_test.to_json(OUTPUT_PATH + 'fever_test_wr.jsonl', orient='records', lines=True)
print('output files created')

In [ ]:
wr_augmented_fever_train

,id,old_premise,augmented_premise,hypothesis,label,wsd,srl
